In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os
import zipfile
from IPython.display import FileLink

# Initial setup
output_dir = "pdf_plots"
os.makedirs(output_dir, exist_ok=True)

# Redshift values
z_values = [0, 0.5, 1, 2, 3]

# Models and their neutrino masses
models = [
    ('fiducial', 'Fiducial', '0 eV'),
    ('Mnu_p', 'Mnu_p', '0.1 eV'),
    ('Mnu_pp', 'Mnu_pp', '0.2 eV'),
    ('Mnu_ppp', 'Mnu_ppp', '0.4 eV')
]

def safe_load_data(file_path):
    """Safe data loading with error handling"""
    try:
        if not os.path.exists(file_path):
            print(f"Warning: File not found - {file_path}")
            return None, None
        data = np.loadtxt(file_path)
        if data.size == 0:
            print(f"Warning: Empty file - {file_path}")
            return None, None
        return data[:, 0], data[:, 1]
    except Exception as e:
        print(f"Error loading {file_path}: {str(e)}")
        return None, None

# Generate all plots
for model_key, model_name, neutrino_mass in models:
    # Combined plot for all z values
    plt.figure(figsize=(10, 6))
    
    for z in z_values:
        file_name = f'/home/jovyan/Data/PDF/matter/{model_key}/0/PDF_m_10.0_z={z}.txt'
        delta, pdf = safe_load_data(file_name)
        if delta is None:
            continue
        plt.plot(delta, pdf, label=f'z={z}')
    
    plt.xscale('symlog', linthresh=0.1)
    plt.yscale('linear')
    plt.xlabel('Overdensity $\delta$')
    plt.ylabel('PDF')
    plt.title(f'Matter PDF ({model_name}, {neutrino_mass})')
    plt.grid(True, which="both", ls="--")
    plt.legend()
    
    plt.savefig(os.path.join(output_dir, f"PDF-{model_key}.jpg"), 
               format='jpg', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Individual plots for each z value
    for z in z_values:
        plt.figure(figsize=(10, 6))
        file_name = f'/home/jovyan/Data/PDF/matter/{model_key}/0/PDF_m_10.0_z={z}.txt'
        delta, pdf = safe_load_data(file_name)
        if delta is None:
            continue
            
        plt.plot(delta, pdf)
        plt.xscale('symlog', linthresh=0.1)
        plt.yscale('linear')
        plt.xlabel('Overdensity $\delta$')
        plt.ylabel('PDF')
        plt.title(f'Matter PDF ({model_name}, {neutrino_mass}, z={z})')
        plt.grid(True, which="both", ls="--")
        
        info_text = f"Z={z}\n{model_name}: {neutrino_mass}"
        plt.text(0.95, 0.95, info_text, transform=plt.gca().transAxes, 
                fontsize=8, va='top', ha='right',
                bbox=dict(boxstyle="round,pad=0.1", edgecolor="black", facecolor="white"))
        
        plt.savefig(os.path.join(output_dir, f"PDF-{model_key}-z{z}.jpg"), 
                   format='jpg', dpi=300, bbox_inches='tight')
        plt.close()

# Model comparison plots
for z in z_values:
    plt.figure(figsize=(10, 6))
    ref_file = f'/home/jovyan/Data/PDF/matter/fiducial/0/PDF_m_10.0_z={z}.txt'
    delta_ref, pdf_ref = safe_load_data(ref_file)
    if delta_ref is None:
        continue
    peak_pdf = np.max(pdf_ref)
    
    for model_key, model_name, neutrino_mass in models:
        file_name = f'/home/jovyan/Data/PDF/matter/{model_key}/0/PDF_m_10.0_z={z}.txt'
        delta, pdf = safe_load_data(file_name)
        if delta is None:
            continue
        plt.plot(delta, pdf, label=f'{model_name} ({neutrino_mass})')
    
    # Symmetric Gaussian for comparison
    delta_pos = np.logspace(-2, 2, 500)
    pdf_pos = norm.pdf(np.log(delta_pos), loc=0, scale=1)
    delta_neg = -np.logspace(-2, 2, 500)[::-1]
    pdf_neg = norm.pdf(np.log(-delta_neg), loc=0, scale=1)
    delta_gauss = np.concatenate([delta_neg, delta_pos])
    pdf_gauss = np.concatenate([pdf_neg, pdf_pos])
    pdf_gauss = pdf_gauss * peak_pdf / np.max(pdf_gauss)
    plt.plot(delta_gauss, pdf_gauss, 'k--', label='Symmetric Log-Gaussian')
    
    plt.xscale('symlog', linthresh=1e-2)
    plt.xlim(-1e2, 1e2)
    plt.yscale('linear')
    plt.xlabel('Overdensity $\delta$')
    plt.ylabel('PDF')
    plt.title(f'Matter PDF Comparison (z={z})')
    plt.grid(True, which="both", ls="--")
    plt.legend()
    
    plt.savefig(os.path.join(output_dir, f"PDF-CompareModels-z{z}.jpg"), 
               format='jpg', dpi=300, bbox_inches='tight')
    plt.close()

# Create zip archive of all output files
zip_filename = "output_plots.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            zipf.write(os.path.join(root, file), 
                      os.path.relpath(os.path.join(root, file), 
                                    os.path.join(output_dir, '..')))

# Create download link in Jupyter
display(FileLink(zip_filename))

/home/jovyan/output_plots.zip